In [1]:
from __future__ import division
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.layers import dropout

In [2]:
#load data
daysActivities = pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/r相关函数/normaldnn/save/daysActivities.csv',
                      header=0,encoding='gbk' )

In [3]:
trainNums =int(len(daysActivities)*0.9)
trainData = daysActivities[:trainNums]
testData = daysActivities[trainNums:]


In [4]:
n_inputs = len(daysActivities.columns)
n_hidden1 = n_inputs//3
n_hidden2 = n_hidden1//3
n_hidden3 = n_hidden2//3
n_hidden4 = n_hidden2
n_hidden5 = n_hidden1
n_outpus = n_inputs

learning_rate=0.0001

In [6]:
activation = tf.nn.elu
initializer = tf.contrib.layers.variance_scaling_initializer()
is_training = tf.placeholder_with_default(False, shape=(), name='is_training')
keep_prob = 0.7

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
X_drop = dropout(X, keep_prob, is_training=is_training)

weights1_init = initializer([n_inputs, n_hidden1])
weights2_init = initializer([n_hidden1, n_hidden2])
weights3_init = initializer([n_hidden2, n_hidden3])

weights1 = tf.Variable(weights1_init, dtype=tf.float32, name="weights1")
weights2 = tf.Variable(weights2_init, dtype=tf.float32, name="weights2")
weights3 = tf.Variable(weights3_init, dtype=tf.float32, name="weights3")

weights4 = tf.transpose(weights3, name="weights4") # tied weights
weights5 = tf.transpose(weights2, name="weights5") # tied weights
weights6 = tf.transpose(weights1, name="weights6") # tied weights

biases1 = tf.Variable(tf.zeros(n_hidden1), name="biases1")
biases2 = tf.Variable(tf.zeros(n_hidden2), name="biases2")
biases3 = tf.Variable(tf.zeros(n_hidden3), name="biases3")
biases4 = tf.Variable(tf.zeros(n_hidden4), name="biases4")
biases5 = tf.Variable(tf.zeros(n_hidden5), name="biases5")
biases6 = tf.Variable(tf.zeros(n_outpus),  name="biases6")

hidden1 = activation(tf.matmul(X_drop,  weights1) + biases1)
hidden2 = activation(tf.matmul(hidden1, weights2) + biases2)
hidden3 = activation(tf.matmul(hidden2, weights3) + biases3)
hidden4 = activation(tf.matmul(hidden3, weights4) + biases4)
hidden5 = activation(tf.matmul(hidden4, weights5) + biases5)
outputs = tf.matmul(hidden5, weights6) + biases6

codings = hidden3

loss = tf.reduce_mean(tf.square(outputs - X)) 


optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()

In [10]:
n_epochs = 1000
batch_size = 1000
batch_num = len(daysActivities) // batch_size+1
with tf.Session() as sess:
    init.run()
    train_loss_eval = loss.eval(feed_dict={X: trainData})
    test_loss_eval  = loss.eval(feed_dict={X: testData})
    print(train_loss_eval,test_loss_eval)
    for epoch in range(n_epochs):  
        for batch in range(batch_num):
            X_batch, y_batch = trainData[batch*batch_size:(batch+1)*batch_size],1
            sess.run(training_op, feed_dict={X: X_batch,is_training:True})
        if epoch%10==0:
            train_loss_eval = loss.eval(feed_dict={X: trainData})
            test_loss_eval  = loss.eval(feed_dict={X: testData})
            print(train_loss_eval,test_loss_eval)
    codings_evl = codings.eval(feed_dict={X: daysActivities})

0.0453142 0.046209
0.0311809 0.0317368
0.0157574 0.0160441
0.0111626 0.0114436
0.0100301 0.0103132
0.00936914 0.00963365
0.00855203 0.008785
0.00782745 0.00802922
0.00716423 0.0073463
0.00679245 0.00697067
0.00652792 0.00670228
0.00629882 0.00646775
0.00609254 0.00625728
0.00592058 0.00608264
0.00579508 0.00595311
0.00570036 0.00585451
0.00560776 0.00575756
0.00552935 0.00567429
0.00544428 0.0055843
0.00535046 0.00548453
0.00524098 0.00536811
0.00511715 0.00523772
0.00497836 0.00509284
0.00485274 0.00496081
0.00473786 0.00483995
0.00462626 0.00472458
0.00453244 0.00462666
0.00444751 0.00453892
0.0043508 0.0044399
0.00426655 0.00435297
0.00418745 0.00427193
0.00412309 0.00420576
0.00405608 0.00413691
0.00399894 0.00407854
0.00395865 0.00403701
0.00390951 0.00398509
0.0038718 0.00394772
0.00383192 0.00390682
0.00380607 0.00388182
0.00377289 0.00384864
0.00375432 0.00382774
0.00372582 0.00379904
0.00369686 0.00376998
0.00368442 0.00375745
0.00366111 0.00373459
0.00365072 0.00372235
0.0036

In [9]:
codings_evl.shape

(28675, 8)

In [8]:
codings_evl

array([[-0.25414002, -0.63542044,  0.20520274, ...,  0.1217172 ,
         0.52474529,  0.79465747],
       [-0.3781848 , -0.70458591,  0.14556412, ..., -0.18826151,
         1.42599559, -0.62399226],
       [-0.65532964, -0.44091332, -0.17544419, ...,  0.17870119,
        -0.47779506,  0.77641749],
       ..., 
       [ 0.02384393, -0.73661482,  0.48142409, ...,  0.63410956,
         0.62226313, -0.70211512],
       [ 0.06161764, -0.72671062,  0.47069463, ...,  0.63366085,
         0.57515436, -0.70670128],
       [-0.61378884, -0.38293511, -0.00907707, ...,  0.31803232,
        -0.40581328,  0.54158503]], dtype=float32)

In [ ]:
codings_evl